# Detección de tweets meteorológicos con Bag of Words

In [77]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd

#### Cargo la base de datos clasificada y genero las variables de importancia

In [78]:
tw = pd.read_csv('base_corr.csv')

In [79]:
tw.keys()[-3:]

Index(['Twit', 'Tag', 'Num'], dtype='object')

In [89]:
msj = tw['Twit']
tag = tw['Tag']
num = tw['Num']

In [90]:
msj

0                                         Esta lloviendo
1      quiero ir al centro a la mañana pero seguro ll...
2                   @paloma_CAI7 Si, llueve y mucho !! 😒
3                       @rozalazar9 Si llueve les hago 🤗
4      Hasta cuándo le voy a seguir teniendo miedo a ...
5      13h ClimaRafaela: Temp. 20.9 °C, Hum. 48 %, Ba...
6      @Eze2450 Sea la estación que sea , no hay plat...
7      Llueve y no tengo Internet...me espera tremend...
8                            Que linda noche de lluvia 💖
9      Miércoles de mucho calor, previo a las lluvias...
10     Me gusta cuando llueve porque parece que se es...
11     Noviembre sin ti es sentir que la lluvia me di...
12     17h Agvnet CDU: Temp. 32.4 °C, Hum. 33 %, Bar....
13      Con esta lluvia me dieron ganas de cucharear ! 😏
14                               Y se largo la lluvia ☔💕
15                                  Que hermosa tormenta
16                         @joaquinv17 dan lluvia jajaja
17     Aseguran que hay stock s

#### Elimino las cosas innecesarias de msj

In [178]:
def normalizacion(mensaje):
    mensaje = list(mensaje)
    lenght = len(mensaje)
    lista_ascii = [['\\xe1', '\\xe9', '\\xed', '\\xf3', '\\xfa', '\\xf1', '\\xd1', '\\xc1','\\xc9', '\\xcd', '\\xd3', 
                 '\\xda', '\\xa1', '\\xbf', '\\xdc', '\\xfc', '\\xb4', '\\xba', '\\xaa', '\\xb7','\\xc7','\\xe7',
                 '\\xa8'],['á', 'é', 'í', 'ó', 'ú', 'ñ', 'Ñ', 'Á', 'É', 'Í', 'Ó', 'Ú', '¡', '¿', 'Ü', 'ü', '´', 
                           'º', 'ª', '·','Ç','ç','¨']]
    for i in range(lenght):
        mensaje[i] = mensaje[i].replace('!','').replace('?','')
        mensaje[i] = mensaje[i].encode('unicode-escape').decode('ASCII')
    for i in range(lenght):
        mensaje[i] = mensaje[i].replace('\\xe1', 'á').replace('\\xf1', 'ñ').replace('\\xa1', '').replace('\\xbf', '').replace('\\xc1', 'Á')
    return(mensaje)

In [132]:
normalizacion(msj)

['Esta lloviendo',
 'quiero ir al centro a la mañana pero seguro llueve jajajajaja',
 '@paloma_CAI7 Si, llueve y mucho  \\U0001f612',
 '@rozalazar9 Si llueve les hago \\U0001f917',
 'Hasta cuándo le voy a seguir teniendo miedo a las tormentas \\U0001f644\\U0001f623',
 '13h ClimaRafaela: Temp. 20.9 \\xb0C, Hum. 48 %, Bar. 1010.1 mbar, Lluvia 0.2 mm, Viento 10.4 km/h SSO #Weathercloud',
 '@Eze2450 Sea la estaci\\xf3n que sea , no hay plata Y ahora llueve.. que te cambia Jajaja ten\\xe9s que estudiar igual',
 'Llueve y no tengo Internet...me espera tremenda tarde',
 'Que linda noche de lluvia \\U0001f496',
 'Mi\\xe9rcoles de mucho calor, previo a las lluvias https://t.co/eFZV0fhc58',
 'Me gusta cuando llueve porque parece que se estan fritando empanadas #PuesGorda \\U0001f602',
 'Noviembre sin ti es sentir que la lluvia me dice llorando que todo acab\\xf3 \\U0001f3b5',
 '17h Agvnet CDU: Temp. 32.4 \\xb0C, Hum. 33 %, Bar. 997.3 hPa, Lluvia 0 mm, Viento 11.2 km/h SE #Weathercloud',
 'Con es

In [138]:
lista_ascii = [['bla', 'ja'],['blabla', 'jajaj']]

In [139]:
lista_ascii

[['bla', 'ja'], ['blabla', 'jajaj']]

In [143]:
lista_ascii[1][1]

'jajaj'

In [172]:
A= 'á, é, í, ó, ú, ñ, Ñ, Á, É, Í, Ó, Ú, ¡, ¿, Ü, ü, ´, º, ª, ·,Ç,ç,¨'

In [173]:
A.encode('unicode-escape').decode('ASCII')

'\\xe1, \\xe9, \\xed, \\xf3, \\xfa, \\xf1, \\xd1, \\xc1, \\xc9, \\xcd, \\xd3, \\xda, \\xa1, \\xbf, \\xdc, \\xfc, \\xb4, \\xba, \\xaa, \\xb7,\\xc7,\\xe7,\\xa8'

In [176]:
 lista_ascii = [['\\xe1', '\\xe9', '\\xed', '\\xf3', '\\xfa', '\\xf1', '\\xd1', '\\xc1','\\xc9', '\\xcd', '\\xd3', 
                 '\\xda', '\\xa1', '\\xbf', '\\xdc', '\\xfc', '\\xb4', '\\xba', '\\xaa', '\\xb7','\\xc7','\\xe7',
                 '\\xa8'],['á', 'é', 'í', 'ó', 'ú', 'ñ', 'Ñ', 'Á', 'É', 'Í', 'Ó', 'Ú', '¡', '¿', 'Ü', 'ü', '´', 
                           'º', 'ª', '·','Ç','ç','¨'']]